In [2]:
from functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/tests/area_1_parameters/sampling_shape/run_1762356356")
vox_path = Path("../data/voxel_maps/area_1/")
pcd = load_point_cloud(pcd_path)
# Load chen skeleton graph
chen_skeleton_graph, id_map_old = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")
id_map = np.load(chen_graph_path_input / "chen_voxel_id_map.npy")
id_map_full = np.load(chen_graph_path_input / "chen_id_map_full.npy")
dist_map = np.load(chen_graph_path_input / "chen_dist_map.npy")
M = np.load(vox_path / "area_1_size_0_1_M.npy")

visualize([VisObjGraph(chen_skeleton_graph), VisObjPCD(pcd), VisObjVoxel(id_map, M)])

voxel_grid_flooded= np.load(vox_path / "area_1_size_0_1_voxel_grid_flooded.npy")
visualize([VisObjVoxel(id_map_full, M)])
visualize([VisObjVoxel(dist_map, M)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Loading graph from ../output/tests/area_1_parameters/sampling_shape/run_1762356356/node_list.txt and ../output/tests/area_1_parameters/sampling_shape/run_1762356356/edge_list.txt...
Loaded graph with 1699 nodes and 1834 edges.


In [13]:
from skimage.draw import line_nd
# inverse transformation matrix
M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    # compute line voxels between v_pos_1 and v_pos_2
    line_voxels = np.array(line_nd(tuple(v_pos_1.astype(int)), tuple(v_pos_2.astype(int)))).T[1:] # skip the first entry, hence its the node itself
    for voxel in line_voxels:
        if voxel_grid[tuple(voxel)] == 1:
            outlaiers.add(edge)
print(f"Total outlier nodes: {len(outlaiers)}")

Total outlier nodes: 10


In [13]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd")
pcd = load_point_cloud(pcd_path)
bounding_box = np.array([[-26.0, -8.0, -1.0], [8.0, 50.0, 3.3]])
start_pt = np.array([-5.0, 9.0, 1.1])
visualize(VisObjPCD(pcd, bounding_box=bounding_box, start_pt=start_pt))

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd with 495103 points.
Adding bounding box to visualization...
Bounding box min: [-26.  -8.  -1.], max: [ 8.  50.   3.3]
Adding start point to visualization...


In [ ]:
from functions import *
import numpy as np

pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd")
pcd = load_point_cloud(pcd_path)
bounding_box = np.array([[-26.0, -8.0, -1.0], [8.0, 50.0, 3.3]])

voxel_grid_inv = np.load("../data/voxel_maps/area_1/area_1_size_0_1_voxel_grid_inverted.npy")
# get indices of occupied voxels (shape: (3, N))
indices = np.array(np.where(voxel_grid_inv == 1))
population = indices.shape[1]

if population < 2:
    raise ValueError(f"Not enough voxels available to sample pairs: found {population}")

# choose up to 200 samples, ensure it's even so we can form pairs
n_samples = min(200, population)
if n_samples % 2 == 1:
    n_samples -= 1
if n_samples <= 0:
    raise ValueError(f"Number of samples computed to be {n_samples}; need at least 2.")

# sample column indices and build (n_samples, 3) array of voxel indices
chosen_cols = np.random.choice(population, n_samples, replace=False)
random_indices = indices[:, chosen_cols].T  # shape (n_samples, 3)

# convert voxel indices to world coordinates
M = np.load("../data/voxel_maps/area_1/area_1_size_0_1_M.npy")
idx_h = np.hstack((random_indices, np.ones((random_indices.shape[0], 1))))
world_coords = (idx_h @ M.T)[:, :3]
voxel_size = M[0, 0]  # assuming uniform scaling
world_coords_center = world_coords + voxel_size / 2.0

# pair points (n_samples must be even)
point_pairs = world_coords_center.reshape(-1, 2, 3)
point_pairs = point_pairs.reshape(-1, 6)

point_pairs_indices = random_indices.reshape(-1, 2, 3)
point_pairs_indices = point_pairs_indices.reshape(-1, 6)

for pair in point_pairs:
    p1 = pair[:3]
    p2 = pair[3:]
    print(f"Point 1: {p1}, Point 2: {p2}")
    print(f"distance: {np.linalg.norm(p2 - p1)}")
    #visualize(VisObjPCD(pcd, bounding_box=bounding_box, start_pt=p1, end_pt=p2))

#np.save("../data/voxel_maps/area_1/area_1_size_0_1_point_pairs_world_coords.npy", point_pairs)
#np.save("../data/voxel_maps/area_1/area_1_size_0_1_point_pairs_voxel_indices.npy", point_pairs_indices)

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd with 495103 points.
Point 1: [-12.22299919  24.88299999   2.627     ], Point 2: [-12.82299919   1.78299999   2.227     ]
distance: 23.111252670506634
Point 1: [-12.82299919  24.58299999   0.327     ], Point 2: [-14.22299919  11.18299999   1.727     ]
distance: 13.545478950557635
Point 1: [-3.52299919  9.98299999  1.427     ], Point 2: [-9.82299919 23.08299999  2.827     ]
distance: 14.603424255975034
Point 1: [-17.72299919  25.98299999   1.727     ], Point 2: [-11.22299919  11.08299999   2.127     ]
distance: 16.260996279441187
Point 1: [-19.62299919   8.18299999   1.127     ], Point 2: [-14.72299919  21.38299999   2.627     ]
distance: 14.159802258506296
Point 1: [-19.32299919  42.08299999   1.927     ], Point 2: [-0.62299919 18.98299999  1.427     ]
distance: 29.724568962392034
Point 1

In [31]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/tests/area_1_parameters/height/run_1762276555")
vox_path = Path("../data/voxel_maps/area_1/")
pcd = load_point_cloud(pcd_path)
# Load chen skeleton graph
chen_skeleton_graph, id_map_old = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")
id_map = np.load(chen_graph_path_input / "chen_voxel_id_map.npy")
M = np.load(vox_path / "area_1_size_0_1_M.npy")

visualize([VisObjGraph(chen_skeleton_graph), VisObjPCD(pcd), VisObjVoxel(id_map, M)])

shortest_path_graph, placeholder = load_graph(chen_graph_path_input / "shortest_path" / "nodes.txt", chen_graph_path_input / "shortest_path" / "edges.txt")

visualize([VisObjGraph(shortest_path_graph), VisObjPCD(pcd), VisObjVoxel(id_map, M)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Loading graph from ../output/tests/area_1_parameters/height/run_1762276555/node_list.txt and ../output/tests/area_1_parameters/height/run_1762276555/edge_list.txt...
Loaded graph with 2120 nodes and 2457 edges.
Loading graph from ../output/tests/area_1_parameters/height/run_1762276555/shortest_path/nodes.txt and ../output/tests/area_1_parameters/height/run_1762276555/shortest_path/edges.txt...
Loaded graph with 49 nodes and 48 edges.


In [32]:
x = np.array([[1, 2, 3, 7], [4, 5, 6, 9]])

In [30]:
dict(zip(x[0], x[1])).get(723,25)
x[np.array([0,2])]

IndexError: index 2 is out of bounds for axis 0 with size 2

In [29]:
from functions import *
pcd_path = Path("../../datasets/datasets/sort/ELAPHS-94M.ply")
pcd = load_point_cloud(pcd_path)
visualize(VisObjPCD(pcd))

Loading point cloud from ../../datasets/datasets/sort/ELAPHS-94M.ply...
Loaded point cloud from ../../datasets/datasets/sort/ELAPHS-94M.ply with 94465067 points.


In [33]:
tuple(x[0])

(np.int64(1), np.int64(2), np.int64(3), np.int64(7))

In [34]:
n26_offsets = np.array([[i, j, k] for i in [-1, 0, 1] for j in [-1, 0, 1] for k in [-1, 0, 1] if not (i == 0 and j == 0 and k == 0)])
n26_weights = np.linalg.norm(n26_offsets, axis=1)

In [35]:
n26_offsets

array([[-1, -1, -1],
       [-1, -1,  0],
       [-1, -1,  1],
       [-1,  0, -1],
       [-1,  0,  0],
       [-1,  0,  1],
       [-1,  1, -1],
       [-1,  1,  0],
       [-1,  1,  1],
       [ 0, -1, -1],
       [ 0, -1,  0],
       [ 0, -1,  1],
       [ 0,  0, -1],
       [ 0,  0,  1],
       [ 0,  1, -1],
       [ 0,  1,  0],
       [ 0,  1,  1],
       [ 1, -1, -1],
       [ 1, -1,  0],
       [ 1, -1,  1],
       [ 1,  0, -1],
       [ 1,  0,  0],
       [ 1,  0,  1],
       [ 1,  1, -1],
       [ 1,  1,  0],
       [ 1,  1,  1]])

In [36]:
n26_weights

array([1.73205081, 1.41421356, 1.73205081, 1.41421356, 1.        ,
       1.41421356, 1.73205081, 1.41421356, 1.73205081, 1.41421356,
       1.        , 1.41421356, 1.        , 1.        , 1.41421356,
       1.        , 1.41421356, 1.73205081, 1.41421356, 1.73205081,
       1.41421356, 1.        , 1.41421356, 1.73205081, 1.41421356,
       1.73205081])

In [6]:
import numpy as np
np.floor(np.array([0.74580345]) / 0.2) * 0.2

array([0.6])